In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os, os.path
import copy
from torch.utils.data.sampler import Sampler
import sklearn
from sklearn.metrics import confusion_matrix
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
from scipy import interp
plt.ion()

#Code adapted from: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [2]:
# Hyperparameters
num_epochs = 20
num_classes = 4
batch_size = 6
learning_rate = 0.001
nc = 3
optimizer = "SGD" #or Adam or RMSprop
imgSize = 224

data_dir = '../pictures'

for x in os.listdir(data_dir):
    print(x)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torchvision.models.densenet161(pretrained=True) #https://pytorch.org/hub/pytorch_vision_densenet/
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, num_classes) #initialize and reshape densenet network

model = model.to(device)

#Load just the state_dict from the trained model
model_conv.load_state_dict(torch.load('../state_dict_densenet_classifier1.pt', map_location=torch.device('cpu')))

In [7]:
#make a custom Dataset

#Custom dataset to get image path https://gist.github.com/andrewjong/6b02ff237533b3b2c554701fb53d5c4d
class ImageFolderWithPaths(datasets.ImageFolder):

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

testdata_transforms = {
    x: transforms.Compose([
        transforms.CenterCrop(imgSize),
        transforms.ToTensor(),
        transforms.Normalize([0.5894, 0.5352, 0.5669], [0.0749, 0.0701, 0.0634]) #mean and sd of training images
    ])
    for x in os.listdir(testdata_dir)
}

#image_datasets = {x: datasets.ImageFolder(os.path.join(testdata_dir, x), testdata_transforms[x]) 
image_datasets = {x: ImageFolderWithPaths(os.path.join(data_dir, x), testdata_transforms[x]) 
                  for x in os.listdir(testdata_dir)}

testdataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4)
              for x in os.listdir(data_dir)}

dataset_sizes = {x: len(image_datasets[x]) for x in os.listdir(data_dir)}
print(dataset_sizes)

model.eval()

plt.ioff()
plt.show()

{'423_P0_sorted': 374, '548_P0_not_sorted': 76, '548_SVF_not_sorted': 632, '499_SVF_not_sorted': 535, '606_SVF_sorted': 249, '607_SVF_not_sorted': 479, '605_sorted_SVF': 296, '607_P0_not_sorted': 65, '605_SVF_not_sorted': 470, '423_SVF_Sorted': 277, '607_SVF_sorted': 252, '606_P0_not_sort': 427, '605_P0_not_sorted': 355, '605_P0_sorted': 350, '423_SVF_not_sorted': 386, '499_P0_sorted': 373, '499_SVF_sorted': 326, '606_SVF_not_sorted': 350, '607_P0_sorted': 206, '548_SVF_sorted': 446, '606_P0_sorted': 354}


In [9]:
#Predictions saved in CSV files
preds = np.zeros(shape = (0,2))

with torch.no_grad():
    for x in os.listdir(data_dir):
        print(str(x))
        preds = np.zeros((0,2))
        for images, labels, paths in testdataloaders[x]:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            preds_batch = np.concatenate((np.array(paths).reshape(len(paths),1), np.array(predicted).reshape(len(predicted),1)), axis = 1)
            preds = np.concatenate((preds, preds_batch), axis = 0)

        np.savetxt('../results/' + str(x) + '.csv', preds, delimiter=',', fmt='%s')
        print("Predicted MBS:", np.mean(preds[:,1].astype(np.float))*2)


423_P0_sorted
Predicted MBS: 2.72192513368984
548_P0_not_sorted
Predicted MBS: 3.8684210526315788
548_SVF_not_sorted


KeyboardInterrupt: 